# Flight Price Analysis
### Q: What is the optimal number of days before departure to book a flight for the lowest fare?
This notebook contains EDA and cleaning for the [Flight Prices](https://www.kaggle.com/datasets/dilwong/flightprices?utm_source=chatgpt.com) data set from Kaggle. The data is being explored and cleaned so it can be used to create a dashboard that answers the question above.

## EDA
The Kaggle data set was well documented and answered several questions that would have otherwise been explored during EDA. In addition to answering some basic questions about the data like size and data type, the EDA aimed to answer the following questions:
- What columns can be dropped to reduce the dataset size?
- How can the data be filtered to reduce the dataset size?
- Which columns would be most useful as filters to enhance insight into the optimal number of days before departure to book a flight?

Info from Kaggle:
- Expedia data from 4/16/22 to 10/5/22
- Data includes 16 US airports
- 31.09 GB
- 27 Columns: 16 string, 3 integer, 3 boolean, 5 other
- 5,999,739 unique values in flight ID
- [Column definitions](https://www.kaggle.com/datasets/dilwong/flightprices?utm_source=chatgpt.com)

Additional EDA questions that arose during exploration
- Why are there some base fares that are under $1.00?



In [73]:
# import data 10% of data, try to find columns to remove
import pandas as pd
eda_df = pd.read_csv('itineraries.csv', nrows=600000)

eda_df.head()


,legId,searchDate,flightDate,startingAirport,destinationAirport,fareBasisCode,travelDuration,elapsedDays,isBasicEconomy,isRefundable,...,segmentsArrivalTimeEpochSeconds,segmentsArrivalTimeRaw,segmentsArrivalAirportCode,segmentsDepartureAirportCode,segmentsAirlineName,segmentsAirlineCode,segmentsEquipmentDescription,segmentsDurationInSeconds,segmentsDistance,segmentsCabinCode
0,9ca0e81111c683bec1012473feefd28f,2022-04-16,2022-04-17,ATL,BOS,LA0NX0MC,PT2H29M,0,False,False,...,1650223560,2022-04-17T15:26:00.000-04:00,BOS,ATL,Delta,DL,Airbus A321,8940,947,coach
1,98685953630e772a098941b71906592b,2022-04-16,2022-04-17,ATL,BOS,LA0NX0MC,PT2H30M,0,False,False,...,1650200400,2022-04-17T09:00:00.000-04:00,BOS,ATL,Delta,DL,Airbus A321,9000,947,coach
2,98d90cbc32bfbb05c2fc32897c7c1087,2022-04-16,2022-04-17,ATL,BOS,LA0NX0MC,PT2H30M,0,False,False,...,1650218700,2022-04-17T14:05:00.000-04:00,BOS,ATL,Delta,DL,Boeing 757-200,9000,947,coach
3,969a269d38eae583f455486fa90877b4,2022-04-16,2022-04-17,ATL,BOS,LA0NX0MC,PT2H32M,0,False,False,...,1650227460,2022-04-17T16:31:00.000-04:00,BOS,ATL,Delta,DL,Airbus A321,9120,947,coach
4,980370cf27c89b40d2833a1d5afc9751,2022-04-16,2022-04-17,ATL,BOS,LA0NX0MC,PT2H34M,0,False,False,...,1650213180,2022-04-17T12:33:00.000-04:00,BOS,ATL,Delta,DL,Airbus A321,9240,947,coach


In [8]:
eda_df.dtypes

legId                                 object
searchDate                            object
flightDate                            object
startingAirport                       object
destinationAirport                    object
fareBasisCode                         object
travelDuration                        object
elapsedDays                            int64
isBasicEconomy                          bool
isRefundable                            bool
isNonStop                               bool
baseFare                             float64
totalFare                            float64
seatsRemaining                         int64
totalTravelDistance                  float64
segmentsDepartureTimeEpochSeconds     object
segmentsDepartureTimeRaw              object
segmentsArrivalTimeEpochSeconds       object
segmentsArrivalTimeRaw                object
segmentsArrivalAirportCode            object
segmentsDepartureAirportCode          object
segmentsAirlineName                   object
segmentsAi

In [10]:
eda_df.columns

Index(['legId', 'searchDate', 'flightDate', 'startingAirport',
       'destinationAirport', 'fareBasisCode', 'travelDuration', 'elapsedDays',
       'isBasicEconomy', 'isRefundable', 'isNonStop', 'baseFare', 'totalFare',
       'seatsRemaining', 'totalTravelDistance',
       'segmentsDepartureTimeEpochSeconds', 'segmentsDepartureTimeRaw',
       'segmentsArrivalTimeEpochSeconds', 'segmentsArrivalTimeRaw',
       'segmentsArrivalAirportCode', 'segmentsDepartureAirportCode',
       'segmentsAirlineName', 'segmentsAirlineCode',
       'segmentsEquipmentDescription', 'segmentsDurationInSeconds',
       'segmentsDistance', 'segmentsCabinCode'],
      dtype='object')

#### Columns To Remove: Why Remove Them
- fareBasisCode: don't know what this means
- elapsedDays: don't think this impacts price much, usually 0
- segmentsDepartureTimeEpochSeconds: likely impacts price but choosing to not include
- segmentsDepartureTimeRaw: likely impacts price but choosing to not include
- segmentsArrivalTimeEpochSeconds: likely impacts price but choosing to not include
- segmentsArrivalTimeEpochSeconds: likely impacts price but choosing to not include
- segmentsArrivalTimeRaw: likely impacts price but choosing to not include
- segmentsArrivalAirportCode: focusing on departure and destination not layovers
- segmentsDepartureAirportCode: focusing on departure and destination not layovers
- segmentsAirlineCode: using the airline name
- segmentsEquipmentDescription: don't care about the type of plane



In [90]:
#file still to large, re-importing with dask
import dask.dataframe as dd
flight_df = dd.read_csv('itineraries.csv', usecols=['legId', 'searchDate', 'flightDate', 'startingAirport',
       'destinationAirport', 'travelDuration',
       'isBasicEconomy', 'isRefundable', 'isNonStop', 'baseFare', 'totalFare',
       'seatsRemaining', 'totalTravelDistance',
       'segmentsAirlineName', 'segmentsDurationInSeconds',
       'segmentsDistance', 'segmentsCabinCode'], assume_missing=True, dtype_backend="numpy_nullable")


In [91]:
flight_df.head()

,legId,searchDate,flightDate,startingAirport,destinationAirport,travelDuration,isBasicEconomy,isRefundable,isNonStop,baseFare,totalFare,seatsRemaining,totalTravelDistance,segmentsAirlineName,segmentsDurationInSeconds,segmentsDistance,segmentsCabinCode
0,9ca0e81111c683bec1012473feefd28f,2022-04-16,2022-04-17,ATL,BOS,PT2H29M,False,False,True,217.67,248.6,9.0,947.0,Delta,8940,947,coach
1,98685953630e772a098941b71906592b,2022-04-16,2022-04-17,ATL,BOS,PT2H30M,False,False,True,217.67,248.6,4.0,947.0,Delta,9000,947,coach
2,98d90cbc32bfbb05c2fc32897c7c1087,2022-04-16,2022-04-17,ATL,BOS,PT2H30M,False,False,True,217.67,248.6,9.0,947.0,Delta,9000,947,coach
3,969a269d38eae583f455486fa90877b4,2022-04-16,2022-04-17,ATL,BOS,PT2H32M,False,False,True,217.67,248.6,8.0,947.0,Delta,9120,947,coach
4,980370cf27c89b40d2833a1d5afc9751,2022-04-16,2022-04-17,ATL,BOS,PT2H34M,False,False,True,217.67,248.6,9.0,947.0,Delta,9240,947,coach


In [92]:
flight_df.dtypes


legId                        string[pyarrow]
searchDate                   string[pyarrow]
flightDate                   string[pyarrow]
startingAirport              string[pyarrow]
destinationAirport           string[pyarrow]
travelDuration               string[pyarrow]
isBasicEconomy                       boolean
isRefundable                         boolean
isNonStop                            boolean
baseFare                             Float64
totalFare                            Float64
seatsRemaining                       float64
totalTravelDistance                  float64
segmentsAirlineName          string[pyarrow]
segmentsDurationInSeconds    string[pyarrow]
segmentsDistance             string[pyarrow]
segmentsCabinCode            string[pyarrow]
dtype: object

In [4]:
from dask.diagnostics import ProgressBar

In [93]:
with ProgressBar():
    size = flight_df.shape[0].compute()

[########################################] | 100% Completed | 223.55 s


In [94]:
print(size)

82138753


In [95]:
#missing values by column
with ProgressBar():
    percent_missing_values = ((flight_df.isnull().sum() / flight_df.size) * 100).compute()

percent_missing_values

[########################################] | 100% Completed | 331.19 s


legId                        0.000000
searchDate                   0.000000
flightDate                   0.000000
startingAirport              0.000000
destinationAirport           0.000000
travelDuration               0.000000
isBasicEconomy               0.000000
isRefundable                 0.000000
isNonStop                    0.000000
baseFare                     0.000000
totalFare                    0.000000
seatsRemaining               0.000000
totalTravelDistance          0.436459
segmentsAirlineName          0.000000
segmentsDurationInSeconds    0.000000
segmentsDistance             0.061103
segmentsCabinCode            0.000000
dtype: float64

In [9]:
# number of nonstop records
with ProgressBar():
    nonstop_count = flight_df['isNonStop'].value_counts().compute()

nonstop_count

[########################################] | 100% Completed | 241.55 s


isNonStop
True     22066888
False    60071865
Name: count, dtype: Int64

In [10]:
# number of nonstop records
with ProgressBar():
    airport_starting = flight_df['startingAirport'].value_counts().compute()
    airport_destination = flight_df['destinationAirport'].value_counts().compute()

airport_destination

[########################################] | 100% Completed | 263.89 s
[########################################] | 100% Completed | 268.47 s


destinationAirport
ORD    5717699
OAK    3657923
DFW    5957280
DEN    4630696
CLT    5411448
BOS    5801538
EWR    4074698
JFK    4413765
SFO    5586204
ATL    5211169
MIA    5103390
IAD    3312681
DTW    4456269
LAX    8006721
PHL    4703822
LGA    6093450
Name: count, dtype: int64[pyarrow]

In [11]:
airport_starting

startingAirport
ORD    5503476
OAK    3809884
DFW    5674959
DEN    4697143
CLT    5494510
BOS    5883876
EWR    3970797
JFK    4425164
SFO    5706482
ATL    5312028
MIA    4930213
IAD    3464378
DTW    4547052
LAX    8073281
PHL    4726187
LGA    5919323
Name: count, dtype: int64[pyarrow]

#### Exploring the Nonstop Flight Data

In [123]:
#filtering for nonstop flights

nonstop_df = flight_df[flight_df['isNonStop'] == 1]
nonstop_df.head()

,legId,searchDate,flightDate,startingAirport,destinationAirport,travelDuration,isBasicEconomy,isRefundable,isNonStop,baseFare,totalFare,seatsRemaining,totalTravelDistance,segmentsAirlineName,segmentsDurationInSeconds,segmentsDistance,segmentsCabinCode
0,9ca0e81111c683bec1012473feefd28f,2022-04-16,2022-04-17,ATL,BOS,PT2H29M,False,False,True,217.67,248.6,9.0,947.0,Delta,8940,947,coach
1,98685953630e772a098941b71906592b,2022-04-16,2022-04-17,ATL,BOS,PT2H30M,False,False,True,217.67,248.6,4.0,947.0,Delta,9000,947,coach
2,98d90cbc32bfbb05c2fc32897c7c1087,2022-04-16,2022-04-17,ATL,BOS,PT2H30M,False,False,True,217.67,248.6,9.0,947.0,Delta,9000,947,coach
3,969a269d38eae583f455486fa90877b4,2022-04-16,2022-04-17,ATL,BOS,PT2H32M,False,False,True,217.67,248.6,8.0,947.0,Delta,9120,947,coach
4,980370cf27c89b40d2833a1d5afc9751,2022-04-16,2022-04-17,ATL,BOS,PT2H34M,False,False,True,217.67,248.6,9.0,947.0,Delta,9240,947,coach


In [14]:
with ProgressBar():
    airport_starting_ns = nonstop_df['startingAirport'].value_counts().compute()
airport_starting_ns

[########################################] | 100% Completed | 262.78 s


startingAirport
ORD    2058013
OAK      59862
DFW    1651933
DEN    1177454
CLT    1321721
BOS    2050948
EWR    1378710
JFK    1696647
SFO    1326117
ATL    1754889
MIA    1353529
IAD     642803
DTW    1046957
LAX    1917116
PHL     784208
LGA    1845981
Name: count, dtype: int64[pyarrow]

In [15]:
with ProgressBar():
    airline_ns = nonstop_df['segmentsAirlineName'].value_counts().compute()

[########################################] | 100% Completed | 261.96 s


In [16]:
airline_ns

segmentsAirlineName
United               4531426
Alaska Airlines       446001
Frontier Airlines     466926
JetBlue Airways      3578393
American Airlines    7453245
Spirit Airlines       735728
Delta                4855169
Name: count, dtype: int64[pyarrow]

In [24]:
#get the number of nonstop flights
with ProgressBar():
    print(nonstop_df['legId'].unique().compute())


[########################################] | 100% Completed | 294.31 s
0       a63591f2b4df39d809c4a45a3898e122
1       1f48703582bd06ae6a213fcfcf74675a
2       18df042c87803919d25cbbd1961cb725
3       cf4ebd89e2d77dd8f3fa7b11cb9014fc
4       2192110016dc12e590d4e3ff0003f159
                      ...               
1384    23b38ece6081af12bfdf423ed7e74358
1385    43ab62855c5d33c44ed2132c0b27b464
1386    acc6d1830481facc444b1341e4f11a76
1387    eba7d85b5623d9e37d8221d5c7d3f823
1388    802532efaaca82fc7c1d7a6de8426690
Name: legId, Length: 651615, dtype: string


In [145]:
#get the number of records per flight ID
with ProgressBar():
    num_searches = nonstop_df['legId'].value_counts().compute().sort_values()
num_searches

[########################################] | 100% Completed | 460.69 s


legId
00177c3f8bdce8919a0bb2ed3b62852b     1
045916f1507688dbcd5b235284a65e40     1
0c9e7392d6f4639290f83b6b6c0f949e     1
15309cd6a650304662dfff497cab0105     1
17e3894f726cfbf801f4383fd2af68de     1
                                    ..
fdb3c4e88cddfe4f685d6310fefc5169    60
fefbf5795655ea4f6d0feea2e01c4a17    60
fea1345623050a76e9f38ec4e5f924f2    61
6365830250822d8732e98ac28735f7b0    61
b2feca28d8d28bff3fe8b9aae6cc9832    61
Name: count, Length: 651615, dtype: int64[pyarrow]

In [173]:
with ProgressBar():
    legId_counts = nonstop_df.groupby('legId').size().compute()

[########################################] | 100% Completed | 283.04 s


In [174]:
legId_counts_df = legId_counts.reset_index()
legId_counts_df.columns = ['legId', 'count']
legId_counts_df.head()

,legId,count
0,000ed2202d16a2ff763ffb276cbee80b,13
1,00b321198a1381bdfb166043df33a45d,4
2,00fa74ec1f408001e8159681893577c1,15
3,012a2879c94082eb386d571d35c6cb7d,4
4,0132d825dfefd2fa95e117b654639b31,13


In [181]:
ledId_sixty_df = legId_counts_df[legId_counts_df['count'] >= 60]
ledId_sixty_df.shape

(11136, 2)

In [183]:
sixty_ids = list(ledId_sixty_df['legId'])
len(sixty_ids)

11136

In [27]:
#Comparing isBasicEconomy and segmentCabinCode
with ProgressBar():
    Cabin = nonstop_df['segmentsCabinCode'].value_counts().compute()
    Economy_TF = nonstop_df['isBasicEconomy'].value_counts().compute()


Economy_TF

[########################################] | 100% Completed | 224.87 s
[########################################] | 100% Completed | 249.52 s


isBasicEconomy
True      3569724
False    18497164
Name: count, dtype: Int64

In [28]:
Cabin

segmentsCabinCode
first               17602
premium coach       28144
business             9963
coach            22011179
Name: count, dtype: int64[pyarrow]

In [39]:
print(Cabin['coach'] == Economy_TF[0])


False


In [8]:
#do the totalTravelDistance and segmentsDistance columns match
nonstop_df['segmentsDistance'] = nonstop_df['segmentsDistance'].astype('float64')

nonstop_df.dtypes


legId                        string[pyarrow]
searchDate                   string[pyarrow]
flightDate                   string[pyarrow]
startingAirport              string[pyarrow]
destinationAirport           string[pyarrow]
travelDuration               string[pyarrow]
isBasicEconomy                       boolean
isRefundable                         boolean
isNonStop                            boolean
baseFare                             Float64
seatsRemaining                       float64
totalTravelDistance                  float64
segmentsAirlineName          string[pyarrow]
segmentsDurationInSeconds    string[pyarrow]
segmentsDistance                     float64
segmentsCabinCode            string[pyarrow]
dtype: object

In [9]:
nonstop_df['distanceComp'] = nonstop_df['totalTravelDistance'] == nonstop_df['segmentsDistance']
nonstop_df.head()

,legId,searchDate,flightDate,startingAirport,destinationAirport,travelDuration,isBasicEconomy,isRefundable,isNonStop,baseFare,seatsRemaining,totalTravelDistance,segmentsAirlineName,segmentsDurationInSeconds,segmentsDistance,segmentsCabinCode,distanceComp
0,9ca0e81111c683bec1012473feefd28f,2022-04-16,2022-04-17,ATL,BOS,PT2H29M,False,False,True,217.67,9.0,947.0,Delta,8940,947.0,coach,True
1,98685953630e772a098941b71906592b,2022-04-16,2022-04-17,ATL,BOS,PT2H30M,False,False,True,217.67,4.0,947.0,Delta,9000,947.0,coach,True
2,98d90cbc32bfbb05c2fc32897c7c1087,2022-04-16,2022-04-17,ATL,BOS,PT2H30M,False,False,True,217.67,9.0,947.0,Delta,9000,947.0,coach,True
3,969a269d38eae583f455486fa90877b4,2022-04-16,2022-04-17,ATL,BOS,PT2H32M,False,False,True,217.67,8.0,947.0,Delta,9120,947.0,coach,True
4,980370cf27c89b40d2833a1d5afc9751,2022-04-16,2022-04-17,ATL,BOS,PT2H34M,False,False,True,217.67,9.0,947.0,Delta,9240,947.0,coach,True


In [51]:
with ProgressBar():
    Distance_Comp = nonstop_df['distanceComp'].value_counts().compute()

[########################################] | 100% Completed | 267.18 s


In [56]:
with ProgressBar():
    missing_values_ns = ((nonstop_df.isnull().sum() / nonstop_df.size) * 100).compute()

missing_values_ns

[########################################] | 100% Completed | 300.91 s


legId                        0.000000
searchDate                   0.000000
flightDate                   0.000000
startingAirport              0.000000
destinationAirport           0.000000
travelDuration               0.000000
isBasicEconomy               0.000000
isRefundable                 0.000000
isNonStop                    0.000000
baseFare                     0.000000
seatsRemaining               0.000000
totalTravelDistance          0.227442
segmentsAirlineName          0.000000
segmentsDurationInSeconds    0.000000
segmentsDistance             0.227442
segmentsCabinCode            0.000000
distanceComp                 0.000000
dtype: float64

In [60]:
print((Distance_Comp[1]/nonstop_df.size)*100)

0.2274419163281042


#### Exploring Odd Fare Prices
With the help of ChatGPT I determined that the total fare for the airlines and dates used in this data set should be \$25 or above. ChatGPT also mentioned that budget carriers like Spirit and Frontier could potentially have had sales making total fare between 19 and 39 dollars.

A review of the airlines with total fare less than \$25 showed it was only budget airlines (Spirit and Frontier) with the very low total fares. So I concluded I did not need to filter out any data due to unreasonable fare prices.

I also decided to use totalFare instead of baseFare to represent the price of the ticket.

In [66]:
#exploring the values for basefare
with ProgressBar():
    basefare = nonstop_df['baseFare'].value_counts().compute().sort_index()
basefare

[########################################] | 100% Completed | 380.61 s


baseFare
0.01         912
0.41        1052
1.0        16267
1.41         861
2.0         1021
           ...  
4426.04        1
4435.35        4
4444.65        1
4557.21       38
4566.51       25
Name: count, Length: 4953, dtype: Int64

In [97]:
totalfare_min = flight_df['totalFare'].min().compute()
totalfare_min

np.float64(19.59)

In [102]:
with ProgressBar():
    totalfare = flight_df['totalFare'].value_counts().compute().sort_index()
totalfare

[########################################] | 100% Completed | 342.92 s


totalFare
19.59      1357
22.47        48
23.97      1465
24.59      6121
25.59        32
           ... 
7554.2        4
7568.6        4
7574.2        1
7918.6        8
8260.61       5
Name: count, Length: 59104, dtype: Int64

In [115]:
totalfare2 = flight_df[flight_df['totalFare'] < 25.00]
totalfare2.tail()

,legId,searchDate,flightDate,startingAirport,destinationAirport,travelDuration,isBasicEconomy,isRefundable,isNonStop,baseFare,totalFare,seatsRemaining,totalTravelDistance,segmentsAirlineName,segmentsDurationInSeconds,segmentsDistance,segmentsCabinCode
94443,3d1afeeff59cbe0abcb75bb3ade5db7f,2022-10-05,2022-11-05,LAX,OAK,PT1H21M,False,False,True,6.0,24.59,0.0,NaN,Spirit Airlines,4860,<NA>,coach
125475,c8e34794697b21f978dcf62a7274efca,2022-10-05,2022-11-08,LAX,OAK,PT1H21M,False,False,True,6.0,24.59,0.0,NaN,Spirit Airlines,4860,<NA>,coach
127726,5e927b894ee77596a9831919e323d16f,2022-10-05,2022-11-08,OAK,LAX,PT1H22M,False,False,True,6.0,24.59,0.0,NaN,Spirit Airlines,4920,<NA>,coach
137736,9962caa59d1f6488e4f6ef6ab9d585e4,2022-10-05,2022-11-09,LAX,OAK,PT1H21M,False,False,True,6.0,24.59,0.0,NaN,Spirit Airlines,4860,<NA>,coach
140017,76664fbc6e9a33c275dbf47d5b91c885,2022-10-05,2022-11-09,OAK,LAX,PT1H22M,False,False,True,6.0,24.59,0.0,NaN,Spirit Airlines,4920,<NA>,coach


In [121]:
len(totalfare2)

8991

In [116]:
with ProgressBar():
    lowfare_airline = totalfare2['segmentsAirlineName'].value_counts().compute()
lowfare_airline

[########################################] | 100% Completed | 229.05 s


segmentsAirlineName
Frontier Airlines    1513
Spirit Airlines      7478
Name: count, dtype: int64[pyarrow]

## Cleaning
Some of the cleaning - like removing columns and filtering for nonstop flights - was done during EDA. The remaining cleaning tasks are outlined below.

Cleaning Tasks
- Remove more unnecessary columns
- Convert searchDate and FlightDate to date objects that can be subtracted from each other
- Calculate daysBeforeDeparture using searchDate and FlightDate
- Convert segmentsDurationInSeconds from string to float64
- Convert segmentsDurationInSeconds to hours
- Limit data to flight IDs with 60 or more records

Columns To Remove: Why Remove Them
- travelDuration: in a weird format, using segmentsDurationInSeconds instead
- isBasicEconomy: doesn't match numbers of class, info in segment class more helpful
- isNonStop: already filtered on this and know data is only for nonstop flights
- segmentsDistance: going to use totalTravelDistance instead
- DistanceComp: column I added that I don't need
- staringAirport: need to save space, get the info from the distance
- destinationAirport: need to save space, get the info from the distance


In [224]:
nonstop_df.head()

,legId,searchDate,flightDate,startingAirport,destinationAirport,travelDuration,isBasicEconomy,isRefundable,isNonStop,baseFare,totalFare,seatsRemaining,totalTravelDistance,segmentsAirlineName,segmentsDurationInSeconds,segmentsDistance,segmentsCabinCode
0,9ca0e81111c683bec1012473feefd28f,2022-04-16,2022-04-17,ATL,BOS,PT2H29M,False,False,True,217.67,248.6,9.0,947.0,Delta,8940,947,coach
1,98685953630e772a098941b71906592b,2022-04-16,2022-04-17,ATL,BOS,PT2H30M,False,False,True,217.67,248.6,4.0,947.0,Delta,9000,947,coach
2,98d90cbc32bfbb05c2fc32897c7c1087,2022-04-16,2022-04-17,ATL,BOS,PT2H30M,False,False,True,217.67,248.6,9.0,947.0,Delta,9000,947,coach
3,969a269d38eae583f455486fa90877b4,2022-04-16,2022-04-17,ATL,BOS,PT2H32M,False,False,True,217.67,248.6,8.0,947.0,Delta,9120,947,coach
4,980370cf27c89b40d2833a1d5afc9751,2022-04-16,2022-04-17,ATL,BOS,PT2H34M,False,False,True,217.67,248.6,9.0,947.0,Delta,9240,947,coach


In [192]:
with ProgressBar():
    nonstop_size = nonstop_df.shape[0].compute()
nonstop_size

[########################################] | 100% Completed | 326.70 s


22066888

In [225]:
nonstop_df2 = nonstop_df[nonstop_df['legId'].isin(sixty_ids)]

In [194]:
with ProgressBar():
    nonstop2_size = nonstop_df2.shape[0].compute()
nonstop2_size

[########################################] | 100% Completed | 450.86 s


668163

In [226]:
#removing columns
nonstop_df2 = nonstop_df2.drop(['travelDuration', 'isBasicEconomy', 'isNonStop', 'segmentsDistance', 'startingAirport', 'destinationAirport'], axis=1)

In [227]:
#Convert flight duration from seconds to hours
nonstop_df2['segmentsDurationInSeconds'] = nonstop_df2['segmentsDurationInSeconds'].astype('float')

nonstop_df2['segmentsDurationInSeconds'] = (nonstop_df2['segmentsDurationInSeconds'] / 3600.0).round(1)

nonstop_df2 = nonstop_df2.rename(columns={'segmentsDurationInSeconds': 'segmentsDurationInHours'})

In [230]:
#checking implementation of segment durtation in hours
nonstop_df2.head(npartitions=-1)

,legId,searchDate,flightDate,isRefundable,baseFare,totalFare,seatsRemaining,totalTravelDistance,segmentsAirlineName,segmentsDurationInHours,segmentsCabinCode
100817,8132c7c2649e99e05bd5fbd68e820f69,2022-05-21,2022-07-18,False,72.0,128.59,0.0,NaN,Spirit Airlines,2.2,coach
122201,8132c7c2649e99e05bd5fbd68e820f69,2022-05-25,2022-07-18,False,112.0,168.59,0.0,NaN,Spirit Airlines,2.2,coach
128331,8132c7c2649e99e05bd5fbd68e820f69,2022-05-26,2022-07-18,False,112.0,168.59,0.0,NaN,Spirit Airlines,2.2,coach
102900,8132c7c2649e99e05bd5fbd68e820f69,2022-05-27,2022-07-16,False,42.0,98.59,0.0,NaN,Spirit Airlines,2.2,coach
101883,8132c7c2649e99e05bd5fbd68e820f69,2022-05-28,2022-07-16,False,42.0,98.59,0.0,NaN,Spirit Airlines,2.2,coach


In [232]:
# Converting date columns to date time objects
nonstop_df2['searchDate'] = nonstop_df2['searchDate'].astype('datetime64[ns]')
nonstop_df2['flightDate'] = nonstop_df2['flightDate'].astype('datetime64[ns]')

nonstop_df2['daysBeforeDeparture'] = nonstop_df2['flightDate'] - nonstop_df2['searchDate']

In [233]:
#checking implemementation of daysBeforeDeparture column
nonstop_df2.head(npartitions=-1)

,legId,searchDate,flightDate,isRefundable,baseFare,totalFare,seatsRemaining,totalTravelDistance,segmentsAirlineName,segmentsDurationInHours,segmentsCabinCode,daysBeforeDeparture
100817,8132c7c2649e99e05bd5fbd68e820f69,2022-05-21,2022-07-18,False,72.0,128.59,0.0,NaN,Spirit Airlines,2.2,coach,58 days
122201,8132c7c2649e99e05bd5fbd68e820f69,2022-05-25,2022-07-18,False,112.0,168.59,0.0,NaN,Spirit Airlines,2.2,coach,54 days
128331,8132c7c2649e99e05bd5fbd68e820f69,2022-05-26,2022-07-18,False,112.0,168.59,0.0,NaN,Spirit Airlines,2.2,coach,53 days
102900,8132c7c2649e99e05bd5fbd68e820f69,2022-05-27,2022-07-16,False,42.0,98.59,0.0,NaN,Spirit Airlines,2.2,coach,50 days
101883,8132c7c2649e99e05bd5fbd68e820f69,2022-05-28,2022-07-16,False,42.0,98.59,0.0,NaN,Spirit Airlines,2.2,coach,49 days


In [234]:
with ProgressBar():
    days_b4 = nonstop_df2['daysBeforeDeparture'].value_counts().compute().sort_index()



[########################################] | 100% Completed | 278.60 s


In [235]:
days_b4

daysBeforeDeparture
1 days     11135
2 days     11136
3 days     11137
4 days     11137
5 days     11136
6 days     11137
7 days     11133
8 days     11137
9 days     11135
10 days    11137
11 days    11137
12 days    11137
13 days    11136
14 days    11140
15 days    11136
16 days    11136
17 days    11136
18 days    11136
19 days    11137
20 days    11136
21 days    11139
22 days    11136
23 days    11136
24 days    11137
25 days    11137
26 days    11136
27 days    11136
28 days    11137
29 days    11136
30 days    11135
31 days    11135
32 days    11136
33 days    11137
34 days    11135
35 days    11136
36 days    11137
37 days    11140
38 days    11135
39 days    11136
40 days    11135
41 days    11132
42 days    11135
43 days    11136
44 days    11136
45 days    11135
46 days    11136
47 days    11136
48 days    11136
49 days    11137
50 days    11137
51 days    11135
52 days    11135
53 days    11136
54 days    11136
55 days    11135
56 days    11135
57 days    11135
58 days    

In [236]:
# removing date columns to reduce dataset size
nonstop_df2 = nonstop_df2.drop(['searchDate'], axis=1)

In [237]:
with ProgressBar():
    airline = nonstop_df2['segmentsAirlineName'].value_counts().compute()
airline

[########################################] | 100% Completed | 261.64 s


segmentsAirlineName
United               138600
Alaska Airlines        8460
Frontier Airlines     15360
JetBlue Airways       96420
American Airlines    246120
Spirit Airlines        7923
Delta                155280
Name: count, dtype: int64[pyarrow]